<a href="https://colab.research.google.com/github/vadlamudi2401/Wumpus_World_/blob/main/Wumpus_World_3410.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install Texttable

In [ ]:
# Import necessary modules
import random  # For generating random positions
from collections import OrderedDict  # To remove duplicate hazard hints in a cell
from texttable import Texttable  # To print the 4x4 grid neatly

# Function to generate a random coordinate (not the starting point [0,0])
def posgen():
    r1 = random.randint(0, 3)  # Random row index
    r2 = random.randint(0, 3)  # Random column index
    while r1 == 0 and r2 == 0:  # Avoid placing hazards/gold at the start
        r1 = random.randint(0, 3)
        r2 = random.randint(0, 3)
    return [r1, r2]

# Generate 5 unique positions for Wumpus, 3 Pits, and Gold
pos = []
while len(pos) != 5:  # Loop until we have 5 unique positions
    t = posgen()
    if t not in pos:
        pos.append(t)

# Initialize an empty 4x4 grid with empty strings
listpos = [["" for _ in range(4)] for _ in range(4)]

# Assign characters to the grid based on random positions
listpos[pos[0][0]][pos[0][1]] = "W"  # Wumpus
listpos[pos[1][0]][pos[1][1]] = "P"  # Pit 1
listpos[pos[2][0]][pos[2][1]] = "P"  # Pit 2
listpos[pos[3][0]][pos[3][1]] = "P"  # Pit 3
listpos[pos[4][0]][pos[4][1]] = "G"  # Gold

# Place the agent at the starting cell (0,0)
listpos[0][0] = "A"

# Function to add environmental indicators (stench/breeze) around hazards
def fn(grid, x, y, condition):
    if 0 <= x < 4 and 0 <= y < 4:  # Check if cell is within bounds
        return grid[x][y] + condition  # Append the condition (S or B)
    else:
        return ""  # Out of bounds, return empty string

# Add 'S' around the Wumpus and 'B' around the Pits
for i in range(0, 4):  # First item is Wumpus, next 3 are pits
    condition = "S" if i == 0 else "B"  # Wumpus causes stench; pits cause breeze
    x, y = pos[i][0], pos[i][1]
    directions = [(x, y+1), (x, y-1), (x+1, y), (x-1, y)]  # Right, Left, Down, Up
    for dx, dy in directions:
        result = fn(listpos, dx, dy, condition)
        if result:
            listpos[dx][dy] = result  # Update cell with condition

# Function to remove duplicate indicators from a cell, preserving order
def removeDupWithOrder(s):
    return "".join(OrderedDict.fromkeys(s))  # E.g., 'SBBS' → 'SB'

# Clean up grid to remove duplicates in each cell
for i in range(4):
    for j in range(4):
        listpos[i][j] = removeDupWithOrder(listpos[i][j])

# Function to draw the grid using Texttable for display
def tabledraw(grid):
    table = Texttable()
    table.add_rows(grid, header=None)  # No headers
    print(table.draw())  # Print the table

# Begin simulation
print("Welcome to Wumpus World!")

# Initialize agent's starting position and tracking variables
i = j = 0  # Agent's position
Expnode = [[0, 0]]  # Visited nodes list
cost = 0  # Total score

# Game loop: run while agent stays within the grid
while (0 <= i < 4) and (0 <= j < 4):
    # Check for indicators in the current cell
    if "S" in listpos[i][j]:
        print("CAME ACROSS SOME STENCH HERE!\n")  # Near Wumpus
    if "B" in listpos[i][j]:
        print("CAME ACROSS SOME BREEZE HERE!\n")  # Near Pit

    print("Possible nodes for traversal:")
    possible = []

    # Check all 4 directions for valid moves
    for dx, dy in [(1, 0), (0, 1), (-1, 0), (0, -1)]:  # Down, Right, Up, Left
        ni, nj = i + dx, j + dy
        if 0 <= ni < 4 and 0 <= nj < 4 and [ni, nj] not in Expnode:
            possible.append([ni, nj])  # Valid move
            print([ni, nj])  # Display move option

    # Get user input to move the agent
    print("Enter your choice for moving ahead!")
    i = int(input("The x component: "))  # Row index
    j = int(input("The y component: "))  # Column index
    cost -= 1  # Movement cost
    Expnode.append([i, j])  # Add to visited nodes

    # Check what's in the new cell
    if "P" in listpos[i][j]:  # Pit encountered
        print("RAN INTO A PIT!!!")
        cost -= 1000
        break
    elif "G" in listpos[i][j]:  # Goal found
        print("FOUND THE GOAL NODE!!! SUCCESS")
        cost += 1000
        break
    elif "W" in listpos[i][j]:  # Wumpus encountered
        print("THE WUMPUS SCREAMS!!!")
        diff = sum(Expnode[-1]) - sum(Expnode[-2])  # Direction of entry

        if diff == -1:  # Approached correctly to kill it
            cost -= 10
            print("THE AGENT SUCCESSFULLY KILLS THE WUMPUS BY CONFRONTATION!!!")
        elif diff == 1:  # Agent got killed
            cost -= 1000
            print("THE AGENT IS DESTROYED BY THE WUMPUS !!! DEFEAT")
            break

# After loop ends — show final state
print("\nFinal Grid:")
tabledraw(listpos)  # Print the final grid

# Show total score
print("\nTotal cost: {}".format(cost))


Welcome to Wumpus World!
CAME ACROSS SOME BREEZE HERE!

Possible nodes for traversal:
[1, 0]
[0, 1]
Enter your choice for moving ahead!
The x component: 0
The y component: 1
Possible nodes for traversal:
[1, 1]
[0, 2]
Enter your choice for moving ahead!
The x component: 1
The y component: 1
CAME ACROSS SOME BREEZE HERE!

Possible nodes for traversal:
[2, 1]
[1, 2]
[1, 0]
Enter your choice for moving ahead!
The x component: 2
The y component: 1
CAME ACROSS SOME BREEZE HERE!

Possible nodes for traversal:
[3, 1]
[2, 2]
[2, 0]
Enter your choice for moving ahead!
The x component: 3
The y component: 1
CAME ACROSS SOME BREEZE HERE!

Possible nodes for traversal:
[3, 2]
[3, 0]
Enter your choice for moving ahead!
The x component: 3
The y component: 2
RAN INTO A PIT!!!

Final Grid:
+----+---+-----+-----+
| AB |   |     |     |
+----+---+-----+-----+
| P  | B | B   |     |
+----+---+-----+-----+
| B  | B | PB  | GSB |
+----+---+-----+-----+
|    | B | PSB | WB  |
+----+---+-----+-----+

Total co